In [1]:
from nltk.tokenize import LineTokenizer, TweetTokenizer, PunktSentenceTokenizer
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Read Word2Vec Model

In [2]:
dictionary = {}
with open('../data/glovetwitter200.txt', 'r') as fd:
    for i, line in enumerate(fd):
        words = line[:-1].split(' ')
        dictionary[words[0]] = array([float(el) for el in words[1:]])
        if i % 100000 == 0:
            print(i)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000


In [8]:
dictionary = pd.Series(dictionary)
dictionary.to_pickle('../data/out/word2vec')

In [11]:
dictionary = pd.read_pickle('../data/out/word2vec')

In [15]:
reverse = dict([(i, j) for i, j in zip(dictionary.keys(), arange(len(dictionary)))])
reverse = pd.Series(reverse)
reverse.to_pickle('../data/out/reverse')

Read Data

In [16]:
data = pd.read_csv('../data/out/data.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [17]:
data.head()

,Hashtags,Origins
0,Photography,Pretty!! :) RT@Cosmopolitan_LV Floral Fusion ...
1,YSL Giveaway,"RT @AHauteMess: FYI friends, my #YSL #Giveaway..."
2,love music technology,#love! RT @Cosmopolitan_LV Interview with the ...
3,FF GIVEAWAY,RT @BELLESnREBELLES: #FF @bikinithief & @fashi...
4,free giveaway,RT @Orbitz: Win a #free round-trip plane ticke...


In [18]:
tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)
exceptions = 0
sync = 0
processed = []

for i, row in enumerate(data.Origins):
    twitt = []
    words = tknzr.tokenize(row)
    for word in words:
        if word[0] == '#':
            continue
            
        if word[0] == '@':
            twitt.append(reverse['<user>'])
            continue
            
        if word[0] == ':':
            twitt.append(reverse['<smile>'])
            continue
            
        try:
            twitt.append(reverse[word.lower()])
        except:
            exceptions += 1
            if exceptions % 10000 == 0:
                print(exceptions, i)
            pass
    processed.append(array(twitt))

10000 7425
20000 14768
30000 22081
40000 30001
50000 37495
60000 44776
70000 51582
80000 59198
90000 66002
100000 73548
110000 80653
120000 87381
130000 94028
140000 102941
150000 110462
160000 118396
170000 126124
180000 132531
190000 140329
200000 148220
210000 155360
220000 163192
230000 171151
240000 179665
250000 187684
260000 194422
270000 200622
280000 206912
290000 214534
300000 220408
310000 227837
320000 235699
330000 243375
340000 250914
350000 259293
360000 267510
370000 274739
380000 281858
390000 288807
400000 295934
410000 304086
420000 309809
430000 317531
440000 325113
450000 332313
460000 340190
470000 347461
480000 356363
490000 364025
500000 372337
510000 379815
520000 388073
530000 396440
540000 405146
550000 412425
560000 418840
570000 426331
580000 433655
590000 441521
600000 449862
610000 458128
620000 465804
630000 473391
640000 480852
650000 488539
660000 496415
670000 504785
680000 512185
690000 519655
700000 527262
710000 535268
720000 543008
730000 550419
7

In [5]:
dictionary = pd.DataFrame({'IndexB': arange(dictionary.index.shape[0]), 
                           'Values': dictionary.values}, index = dictionary.index)

In [19]:
pd.DataFrame({'Hashtags': data.Hashtags,
              'Vecs': processed}).to_pickle('../data/out/processed')